In [ ]:
from requests import get
from json import load, loads
import pandas as pd
from bs4 import BeautifulSoup

# Import all courses name and ID

In [ ]:
courses_dict = load(open('data/courses/all_courses_dict.json'))
list(courses_dict.keys())

# `GET` Analytics Page

In [ ]:
name = 'Data Science masters'

url = f'https://learn.pwskills.com/course-analytics/{name.replace(" ", "-")}/{courses_dict[name]}'
r = get(url)
r

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
script = soup.find('script', {'id': '__NEXT_DATA__'})
data = loads(script.text)    # type: ignore

In [ ]:
users = data['props']['pageProps']['analytics']['users']
submissions = data['props']['pageProps']['analytics']['submissions']
quiz_analytics = data['props']['pageProps']['quizAnalytics']

In [ ]:
df1 = pd.DataFrame(submissions)
df2 = pd.DataFrame(users)
df3 = pd.DataFrame(quiz_analytics)

In [ ]:
df1.shape, df2.shape, df3.shape

## Concatenate

Concatenate `df1` and `df2` along with `axis=1` because they are in same order.

In [ ]:
df = pd.concat([df1, df2], axis=1).drop(columns=['_id'])

In [ ]:
df[df['img'].isnull()==1]

# Import `sections` data

In [ ]:
from utils import LiveCourse

In [ ]:
live_course = {
    "paramLength": 0,
    "sections": data['props']['pageProps']['sections'],
    "courseName": name,
}

In [ ]:
lc = LiveCourse(**live_course).merged_df(courses_dict[name])

In [ ]:
lc.head()

In [ ]:
no_of_assignment = lc.query('type=="assignment"').shape[0]
no_of_quiz = lc.query('type=="quiz"').shape[0]

resource_type_count = lc['type'].value_counts().to_dict()

total_assignment_points = int(lc['totalPointsInAssignment'].sum())
total_quiz_points = int(lc['totalQuestionsInQuiz'].sum())
total_video_duration = int(lc['duration'].sum()/3600)

In [ ]:
resource_type_count

In [ ]:
total_assignment_points, total_quiz_points, total_video_duration

## Course Analytics Analysis

### Top 3

In [ ]:
df[['firstName', 'lastName', 'assignmentsMarkedCount', 'totalAssignmentsScore']].head(3)

### Under 50

In [ ]:
df[['firstName', 'lastName', 'assignmentsMarkedCount', 'totalAssignmentsScore']].loc[3:]

### Top 10 Students in Quiz submission

In [ ]:
df3[['totalPoints', 'firstName', 'lastName']]